## CLM-Demeter integration tutorial


#### This tutorial describes the steps necessary to produce CLM-derived input files for Demeter and execute Demeter using PIC using the created inputs.


#### The files stored to execute the CLM-Demeter experiment can be found here:
`/pic/projects/GCAM/demeter/runs/clm`

#### The following are the associated file paths and additional info to run the code:


In [5]:
import os

project_dir = '/pic/projects/GCAM/demeter/runs/clm'

# file created to spatial represent the base year of 2005 at 0.05 degree for each CLM PFT in units squared-degrees
original_baselayer_file = os.path.join(project_dir, 'inputs', 'observed', 'baselayerdata.csv')

# output file to save the base layer reclassified from CLM to Demeter land classes to
reclassified_baselayer_file = os.path.join(project_dir, 'inputs', 'observed', 'clm_baselayer_by_final_lcs.csv')

# file allocating CLM PFTs to Demeter's final land classes
spatial_allocation_file = os.path.join(project_dir, 'inputs', 'allocation', 'spatial_allocation_rules.csv')

# file allocating GCAM land classes to Demeter's final land classes
projected_allocation_file = os.path.join(project_dir, 'inputs', 'allocation', 'gcam_allocation_rules.csv')

# output allocation file representing reclassification of GCAM land classes to Demeter's land classes
out_proj_allocation_file = os.path.join(project_dir, 'inputs', 'allocation', 'gcam_allocation_rules_reclass.csv')

# output allocation file representing reclassification of CLM PFTs to Demeter's land classes
out_spat_allocation_file = os.path.join(project_dir, 'inputs', 'allocation', 'spatial_allocation_rules_reclass.csv')

# file for land projections from GCAM for all years, subregions, and land classes
projected_file = os.path.join(project_dir, 'inputs', 'projected', 'gcam_ssp4_34_lumip.csv')

# output file to save the disaggregated land projections from GCAM to
out_projected_file = os.path.join(project_dir, 'inputs', 'projected', 'gcam_ssp4_34_lumip_disagg.csv')

# Either 'aez_id' or 'basin_id' depending on the version of GCAM used
metric = 'aez_id'

# list of GCAM years to process
gcam_year_list = [1990] + list(range(2005, 2105, 5))

project_dir_CLM = '/pic/projects/im3/lulc_thrust_area/minchen/Demeter_0.05deg_trial/Demeter_0.05deg_trial'

# original CLM netcdf file
orig_baselayer_netcdf_file = os.path.join(project_dir_CLM, 'mksrf_78pft_landuse_rc2000_c130927.nc')

# ESACCI land cover file
esacci_lu_file = os.path.join(project_dir_CLM, 'ESACCI-LC-aggregated-0.25Deg-merged1992-base.csv')

# Nutrient availability file
nut_avail_file = os.path.join(project_dir_CLM, '000_nutrientavail_hswd_NA_0.25.csv')

# Soil quality file
soil_qual_file = os.path.join(project_dir_CLM, '001_soilquality_hswd_0.25.csv')


#### Import the modules we need from this package:


In [2]:
import clm_demeter.reclassify_projected as rp
import clm_demeter.reclassify_allocation as ra

from clm_demeter.reclassify_base import ReclassBaselayer
from clm_demeter.prepare_baselayer import PrepareBaselayer

### TODO:  Min to insert:
 -  the logic and any code that was used to create the original CLM base layer
 -  what CLM version and assumptions were used to generate the source data
 -  document how the transitions were mapped
 -  document how the treatment order was determined
 -  the GCAM version used in each of the LUMIP SSP4 runs (RCP 3.4 and RCP 6.0)


### Step 0: Create the CLM base layer

**Check out help on our PrepareBaselayer class to see what it requires:**

In [ ]:
help(PrepareBaselayer)

In [ ]:
PrepareBaselayer(orig_baselayer_netcdf_file, esacci_lu_file, nut_avail_file, soil_qual_file)

### Step 1: Reclassify the existing CLM base layer that represents CLM PFTs to Demeter's desired output land classes


#### Check out help on our `ReclassBaselayer` class to see what it requires:

In [17]:
help(ReclassBaselayer)


Help on class ReclassBaselayer in module clm_demeter.reclassify_base:

class ReclassBaselayer(builtins.object)
 |  Reclassify CLM PFTs in the baselayer for Demeter to Demeter final landcover classes.
 |  
 |  :param clm_baselayer_file:              Full path with file name and extension to
 |                                          the CLM baselayer prepared for Demeter
 |                                          in units square-degrees.
 |  
 |  :param clm_spatial_allocation_file:     Full path with file name and extension to
 |                                          the Demeter spatial allocation file for CLM
 |                                          landclasses.
 |  
 |  :param out_clm_baselayer_file:          Full path with file name and extension to
 |                                          to save the reclassified baselayer.
 |  
 |  :method demeter_to_clm_map:             Create a dictionary of Demeter final landclasses
 |                                          to CLM P

#### Generate a reclassififed base layer for Demeter from the original CLM base layer:


In [9]:
base = ReclassBaselayer(original_baselayer_file, spatial_allocation_file, reclassified_baselayer_file)


### Step 2: Disaggregate any projected land class areas per subregion by what is fractionally represented in the base layer.  This prevents Demeter's built-in method from spliting the area evenly regardless of what is in the observed data.


#### Check out the help on the batch function as well as the class that does the work:


In [18]:
help(rp.batch_process_split)


Help on function batch_process_split in module clm_demeter.reclassify_projected:

batch_process_split(projected_allocation_file, observed_baselayer_file, projected_file, out_projected_file, metric, gcam_year_list)
    Batch process projected land class disaggregation.
    
    :param projected_allocation_file:   Full path with file name and extension of the projected allocation file from
                                        Demeter that maps GCAM land classes to the final land cover classes in
                                        Demeter.
    
    :param observed_file:               Full path with file name and extension of the observed data file to be used in the
                                        Demeter run.
    
    :param projected_file:              Full path with file name and extension of the projected data file that has been
                                        extracted from a GCAM output database for use with Demeter.
    
    :param out_projected_file:        

In [19]:
help(rp.GcamLandclassSplit)


Help on class GcamLandclassSplit in module clm_demeter.reclassify_projected:

class GcamLandclassSplit(builtins.object)
 |  Split a GCAM landclass into multiple classes based on the fractional amount present in the observed data per
 |  subregion.  This method is more desirable than the default "even percentage" split that Demeter conducts.  The
 |  output file replaces the GCAM target landclass (e.g. RockIceDesert) with the user-selected classes (e.g. snow and
 |  sparse) per subregion.  The new file becomes what is referenced as the projected file in Demeter.
 |  
 |  :param observed_file:               Full path with file name and extension of the observed data file to be used in the
 |                                      Demeter run.
 |  
 |  :param projected_file:              Full path with file name and extension of the projected data file that has been
 |                                      extracted from a GCAM output database for use with Demeter.
 |  
 |  :param target_lan

#### Run batch disaggregation of land classes:
#### TODO:  Notice in the output that Demeter land class `Urban` is not represented in the CLM base layer. This needs to be addressed in the base layer.


In [6]:
proj = rp.batch_process_split(projected_allocation_file, 
                                reclassified_baselayer_file,
                                projected_file,
                                out_projected_file,
                                metric,
                                gcam_year_list)


Disaggregating projected land class 'Corn' to '['Corn_rf', 'Corn_irr']'
Disaggregating projected land class 'Wheat' to '['Wheat_rf', 'Wheat_irr']'
Disaggregating projected land class 'Rice' to '['Rice_rf', 'Rice_irr']'
Disaggregating projected land class 'Root_Tuber' to '['OtherCrop_rf', 'OtherCrop_irr']'
Disaggregating projected land class 'OilCrop' to '['Soy_rf', 'Soy_irr']'
Disaggregating projected land class 'SugarCrop' to '['Sugarcrop_rf', 'Sugarcrop_irr']'
Disaggregating projected land class 'OtherGrain' to '['OtherCrop_rf', 'OtherCrop_irr']'
Disaggregating projected land class 'FiberCrop' to '['Cotton_rf', 'Cotton_irr']'
Disaggregating projected land class 'FodderGrass' to '['Fodder_rf', 'Fodder_irr']'
Disaggregating projected land class 'FodderHerb' to '['Fodder_rf', 'Fodder_irr']'
Disaggregating projected land class 'biomass' to '['Bioenergy_rf', 'Bioenergy_irr']'
Disaggregating projected land class 'MiscCrop' to '['OtherCrop_rf', 'OtherCrop_irr']'
Disaggregating projected lan

### Step 3:  Reclassify the spatial and projected allocation file


#### Check out help on the two functions we need:


In [21]:
help(ra.reclass_projected_allocation)


Help on function reclass_projected_allocation in module clm_demeter.reclassify_allocation:

reclass_projected_allocation(projected_data_file, projected_allocation_file, out_proj_allocation_file)
    Reclassify the projected allocation file for Demeter to account for classes that have
    been disaggregated to Demeter's final land classes.
    
    :param projected_file:              Full path with file name and extension of the projected data file that has been
                                        extracted from a GCAM output database for use with Demeter.
    
    :param projected_allocation_file:   Full path with file name and extension of the projected allocation file from
                                        Demeter that maps GCAM land classes to the final land cover classes in
                                        Demeter.
    
    :param out_proj_allocation_file:    Full path with file name and extension for the reclassified projected allocation
                          

In [22]:
help(ra.reclass_spatial_allocation)


Help on function reclass_spatial_allocation in module clm_demeter.reclassify_allocation:

reclass_spatial_allocation(spatial_allocation_file, out_spatial_allocation_file, reclassified_baselayer_file)
    Reclassify the spatial allocation file for Demeter to 1:1 land class relationships
    for Demeter. Using the output of this functions assumes that the user has already
    created a reclassified base layer.
    
    :param spatial_allocation_file:     Full path with file name and extension to
                                        the Demeter spatial allocation file for CLM
                                        landclasses.
    
    :param out_spat_allocation_file:    Full path with file name and extension for the reclassified spatial allocation
                                        file.
    
    :param reclassified_baselayer_file: Full path with file name and extension to the reclassifed base layer.



#### Reclass the projected allocation file:


In [3]:
ra.reclass_projected_allocation(out_projected_file, projected_allocation_file, out_proj_allocation_file)


Projected land class 'Jatropha' not in Demeter land classes.


#### Reclass the spatial allocation file:


In [4]:
ra.reclass_spatial_allocation(spatial_allocation_file, out_spat_allocation_file, reclassified_baselayer_file)


Missing land class in spatial base data:  Urban


### Step 4:  Set-up Demeter run

#### Make sure the configuration file is set appropriately (the following uses an example that can be ran on PIC).  This file named `config_ssp4_rcp3p4.ini` is what is called by Demeter to initialize a run.
```ini
[STRUCTURE]
root_dir =                      /pic/projects/GCAM/demeter/runs/clm
in_dir =                        inputs
out_dir =                       outputs

[INPUTS]
allocation_dir =                allocation
observed_dir =                  observed
constraints_dir =               constraints
projected_dir =                 projected
ref_dir =                       reference

[[ALLOCATION]]
spatial_allocation =            spatial_allocation_rules_reclass.csv
gcam_allocation =               gcam_allocation_rules_reclass.csv
kernel_allocation =             kernel_density_allocation_rules.csv
transition_order =              priority_allocation_rules.csv
treatment_order =               treatment_order_allocation_rules.csv
constraints =                   constraints_allocation_rules.csv

[[OBSERVED]]
observed_lu_data =              clm_baselayer_by_final_lcs.csv

[[PROJECTED]]
projected_lu_data =             gcam_ssp4_34_lumip_disagg.csv

[[REFERENCE]]
gcam_regnamefile =              gcam_regions_32.csv
region_coords =                 regioncoord.csv
country_coords =                countrycoord.csv

[OUTPUTS]
diag_dir =                      diagnostics
log_dir =                       log_files
kernel_map_dir =                kernel_density
transition_tabular =            transition_tabular
transition_maps =               transition_maps
luc_intense_p1_dir =            luc_intensification_pass1
luc_intense_p2_dir =            luc_intensification_pass2
luc_expand_dir =                luc_expansion
luc_timestep =                  luc_timestep
lc_per_step_csv =               spatial_landcover_tabular
lc_per_step_nc =                spatial_landcover_netcdf
lc_per_step_shp =               spatial_landcover_shapefile

[[DIAGNOSTICS]]
harm_coeff =                    harmonization_coeff.npy
intense_pass1_diag =            intensification_pass_one_diag.csv
intense_pass2_diag =            intensification_pass_two_diag.csv
expansion_diag =                expansion_diag.csv

[PARAMS]
# projection model name
model =                         GCAM

# projection model metric, currently AEZ or BASIN
metric =                        AEZ

# scenario name
scenario =                      clm-dem-ssp4-rcp34

# run description
run_desc =                      clm-dem-ssp4-rcp34

# aggregate level; 1 if there is no region information in the file, 2 if by both region and AEZ (Default)
agg_level =                     2

# spatial base layer id field name
observed_id_field =             fid

# first year to process
start_year =                    2005

# last year to process
end_year =                      2100

# enter 1 to use non-kernel density constraints, 0 to ignore non-kernel density constraints
use_constraints =               1

# the spatial resolution of the observed spatial data layer in decimal degrees
spatial_resolution =            0.05

# error tolerance in km2 for PFT area change not completed
errortol =                      0.001

# time step in years
timestep =                      5

# factor to multiply the projected land allocation by
proj_factor =                   1000

# output diagnostic reports
diagnostic =                    1

# from 0 to 1; ideal fraction of LUC that will occur during intensification, the remainder will be expansion
intensification_ratio =         0.8

# activates the stochastic selection of grid cells for expansion of any PFT
stochastic_expansion =          1

# threshold above which grid cells are selected to receive a given land type expansion; between 0 and 1, where 0 is all
#     land cells can receive expansion and set to 1 only the grid cell with the maximum likelihood will expand.  For
#     a 0.75 setting, only grid cells with a likelihood >= 0.75 x max_likelihood are selected.
selection_threshold =           0.75

# radius in grid cells to use when computing the kernel density; larger is smoother but will increase run-time
kernel_distance =               30

# create kernel density maps; 1 is True
map_kernels =                   0

# create land change maps per time step per land class
map_luc_pft =                   0

# create land change maps for each intensification and expansion step
map_luc_steps =                 0

# creates maps of land transitions for each time step
map_transitions =               0

# years to save data for, default is all; otherwise a semicolon delimited string e.g, 2005;2050
target_years_output =           all

# save tabular spatial landcover as CSV; define tabular_units below (default sqkm)
save_tabular =                  1

# untis to output tabular data in (sqkm or percent)
tabular_units =                 sqkm

# exports CSV files of land transitions for each time step in km2
save_transitions =              0

# create land cover per point shapefile output for each time step; output units will be same as tabular data
save_shapefile =                0

# create a NetCDF file of land cover percent for each year by grid cell containing each land class
save_netcdf_yr =                0

# create a NetCDF file of land cover percent by land class by grid cell containing each year interpolated to one-year intervals
save_netcdf_lc =                0

# create an ASCII raster representing the land class with the maximum value per grid cell per year
save_ascii_max =                0
```

#### A simple Python code is prepared to run Demeter located here: `/pic/projects/GCAM/demeter/runs/clm/code/run_demeter_ssp4_rcp3p4.py`:
```python
import os

from demeter.model import Demeter


if __name__ == "__main__":

    root = '/pic/projects/GCAM/demeter/runs/clm'

    ssp = 4
    rcp = '3p4'

    print("Processing:  SSP{0} - RCP{1}".format(ssp, rcp))
    ini = os.path.join(root, 'config_ssp{0}_rcp{1}.ini'.format(ssp, rcp))
    dm = Demeter(config=ini)
    dm.execute()
    del dm
```

#### And a shell script to call when submitting the job using PIC located here: `/pic/projects/GCAM/demeter/runs/clm/code/run_demeter_ssp4_rcp3p4.sh`:
```sh

#!/bin/zsh

#SBATCH -n 8
#SBATCH -t 2000
#SBATCH -A IM3
#SBATCH -J clm-demeter-ssp4-rcp3p4

source  /etc/profile.d/modules.sh
module purge
module load python/2.7.13

PFILE="/pic/projects/GCAM/demeter/runs/clm/code/run_demeter_ssp4_rcp3p4.py"

date
python $PFILE
date

echo 'completed'
```

#### After Demeter is installed as a user (see `/pic/projects/GCAM/demeter/README.txt`), you can submit the job by running the following in your terminal:
`sbatch /pic/projects/GCAM/demeter/runs/clm/code/run_demeter_ssp4_rcp3p4.sh`



